In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import astropy.cosmology
from astropy.cosmology import Planck15 as cosmo
import scipy.stats
import time

In [3]:
# MAY NEED ADJUSTING DEPEDNING ON DEVICE/OS RUNNING
data = pd.read_csv("../../Data/3947.csv",comment='#')
data['band'] = data['band'].str.slice(start=2).astype(int)
# Converts 'band' column from type string to int

In [13]:
data.iloc[:1000].to_csv("../../Data/reduced3947.csv",)

In [47]:
data.columns.values
# data.loc[data['ref_id']==5]



array(['ref_id', 'zb', 'odds', 'pz_width', 'zb_mean', 'chi2', 'n_band',
       'ebv', 'qz', 'best_run', 'production_id', 'paudm_id', 'ra', 'dec',
       'zp_gal', 'zl68_gal', 'zu68_gal', 'zl99_gal', 'zu99_gal', 'zp_sec',
       'dchi', 'imag', 'ei', 'i_auto', 'nbfilt', 'mod_gal', 'type',
       'umag', 'bmag', 'vmag', 'gmag', 'rmag', 'zmag', 'icmag', 'jmag',
       'kmag', 'mv', 'ebv_gal', 'ebv_int', 'acs_a_image', 'acs_b_image',
       'acs_theta_image', 'acs_mag_auto', 'acs_magerr_auto', 'zspec',
       'conf', 'f814w', 'zfits', 'r50', 'sersic_n_gim2d', '_col50',
       'band', 'flux', 'flux_error', '_col54', 'n_coadd', 'run', '_col57'],
      dtype=object)

### Creating new data frame of | ref_id_2 | z_photo | z_spec | n_bands |  bandx0 | bandx1 | ... |
(Not all objects have z_spec info)

In [20]:
z_photo = []
z_spec = []

# data_nonan = data.loc[(data['ref_id_2'].notna())&(data['zspec'].notna())&(data['zspec']!=0)]
bands = data['band'].unique()
bands.sort()

# object_ids = data_nonan['ref_id_1'].unique()

truth_table = data['ref_id'].iloc[1:].values != data['ref_id'].iloc[:-1].values
truth_table = np.append(truth_table, [True])
data_sample = data.loc[truth_table,(data.columns != 'flux')&(data.columns != 'band')&(data.columns != 'flux_error')]
object_ids = data_sample['ref_id']
# object_ids, z_photo, z_spec, obj_type, conf = \
#                         data_nonan.loc[truth_table]['ref_id_2'].values,\
#                         data_nonan.loc[truth_table]['zb'].values,\
#                         data_nonan.loc[truth_table]['zspec'].values, \
#                         data_nonan.loc[truth_table]['type'].values, \
#                         data_nonan.loc[truth_table]['conf'].values


band_counts = []
band_data = np.zeros((object_ids.size,bands.size*3+1))
band_data[:,:] = np.nan
# band_data[0,:] = bands[:]

for ind,i in enumerate(object_ids):
    if ind % 1000 == 0: print(ind)
    
    band_counts = data.loc[(data['ref_id'] == i),'band'].count()
#     print(band_counts)
    band_data[ind,-1] = band_counts
    sample = data.loc[(data['ref_id'] == i)].sort_values(by=['band'])
#     print(sample)
    fluxs = sample['flux'].values
    flux_errors = sample['flux_error'].values
    band_sample = sample['band'].values
    
    for e,i in enumerate(bands):
        
        if np.all(i != band_sample): continue
        else:
            band_data[ind,0+e] = band_sample[band_sample == i][:]
            band_data[ind,40+e] = fluxs[band_sample == i][:]
            band_data[ind,80+e] = flux_errors[band_sample == i][:]
        
#     print(band_data)
    
#     fluxs = sample['flux'].values
#     if fluxs.size == 40:
#         band_data[ind+1,:] = fluxs[:]
        
#     else:    
#         for j,b in enumerate(bands):
#             try:
#                 flux = data_nonan.loc[(data_nonan['ref_id_2'] == i)&(data_nonan['band'] == b)]['flux'].values[:]
#                 band_data[ind+1,j] = flux
#             except Exception as e:
#     #             print(i,b,flux)
#     #             print(e)
#                 continue
# print(band_data)
# print(band_data[:,-1])
band_headers = np.array(['band{}'.format(i) for i in range(40)])
flux_headers = np.array(['band{}_flux'.format(i) for i in range(40)])
flux_error_headers = np.array(['band{}_flux_error'.format(i) for i in range(40)])
headers = np.append(band_headers,flux_headers)
headers = np.append(headers,flux_error_headers)
headers = np.append(headers,['n_bands'])

band_data_df = pd.DataFrame(data=band_data[:,:],columns=headers)
# datasetup = {'ref_id_2':object_ids,'z_photo':z_photo,'z_spec':z_spec,'type':obj_type,'conf':conf,'n_bands':band_counts}
data_sample.reset_index(drop=True,inplace=True)
objects_zbs_df = data_sample.join(band_data_df)

# objects_zbs_df.reset_index(drop=True,inplace=True)

objects_zbs_df.to_pickle\
        ("../../Data/Object_zs_zp{}.pkl.gz".format(time.strftime("_%y%m%d_%H%M%S")),"gzip")

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
[40. 40. 40. ... 40. 40. 40.]


In [21]:
# Generating df with object_id - zphoto - zspec
# Has been saved to data folder so can be read from "Object_zs_zp.pkl.gz"

objects_zbs_df = pd.read_pickle("../../Data/Object_zs_zp_181203_130401.pkl.gz")

# objects_zbs_df.loc[objects_zbs_df['ref_id_2'] == '107515']


In [23]:
objects_zbs_df.to_csv("../../Data/reformatted3947_2.csv")

In [22]:
np.savetxt('../../Data/test.txt',objects_zbs_df.values,fmt="%8.3f",newline="\n\n")

In [9]:
band_headers = np.array(['band{}'.format(i) for i in range(40)])
flux_headers = np.array(['band{}_flux'.format(i) for i in range(40)])


array(['band0_flux', 'band1_flux', 'band2_flux', 'band3_flux',
       'band4_flux', 'band5_flux', 'band6_flux', 'band7_flux',
       'band8_flux', 'band9_flux', 'band10_flux', 'band11_flux',
       'band12_flux', 'band13_flux', 'band14_flux', 'band15_flux',
       'band16_flux', 'band17_flux', 'band18_flux', 'band19_flux',
       'band20_flux', 'band21_flux', 'band22_flux', 'band23_flux',
       'band24_flux', 'band25_flux', 'band26_flux', 'band27_flux',
       'band28_flux', 'band29_flux', 'band30_flux', 'band31_flux',
       'band32_flux', 'band33_flux', 'band34_flux', 'band35_flux',
       'band36_flux', 'band37_flux', 'band38_flux', 'band39_flux'],
      dtype='<U11')

In [20]:
data.loc[data['ref_id_2']==90177].loc[:,['flux','band']].shape

(23, 2)